## 内置方式

In [1]:
import numpy as np
import scipy.sparse
import pickle
import xgboost as xgb

In [2]:
dtrain=xgb.DMatrix('data/agaricus.txt.train')
dtest=xgb.DMatrix('data/agaricus.txt.test')

#只在有值的地方有记录，省空间。

[09:32:28] 6513x127 matrix with 143286 entries loaded from data/agaricus.txt.train
[09:32:28] 1611x127 matrix with 35442 entries loaded from data/agaricus.txt.test


In [3]:
param={'max_depth':2,'eta':1,'silent':1,'objective':'binary:logistic'}

In [4]:
watchlist=[(dtest,'eval'),(dtrain,'train')]
num_round=2
bst=xgb.train(param,dtrain,num_round,watchlist)

[0]	eval-error:0.042831	train-error:0.046522
[1]	eval-error:0.021726	train-error:0.022263


In [5]:
preds=bst.predict(dtest)

In [6]:
labels=dtest.get_label()

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [8]:
data =pd.read_csv('data/Pima-Indians-Diabetes.csv')

In [9]:
train,test=train_test_split(data)

In [10]:
feature_columns=['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction']
target_column='Outcome'

In [11]:
xgtrain=xgb.DMatrix(train[feature_columns].values,train[target_column].values)
xgtest=xgb.DMatrix(test[feature_columns].values,test[target_column].values)


In [12]:
param={'max_depth':5,'eta':0.1,'silent':1,'subsample':0.7,'colsample_bytree':0.7,'objective':'binary:logistic'}

In [13]:
watchlist=[(xgtest,'eval'),(xgtrain,'train')]
num_round=10
bst=xgb.train(param,xgtrain,num_round,watchlist)

[0]	eval-error:0.354167	train-error:0.262153
[1]	eval-error:0.286458	train-error:0.197917
[2]	eval-error:0.276042	train-error:0.175347
[3]	eval-error:0.270833	train-error:0.173611
[4]	eval-error:0.265625	train-error:0.161458
[5]	eval-error:0.260417	train-error:0.135417
[6]	eval-error:0.255208	train-error:0.135417
[7]	eval-error:0.255208	train-error:0.138889
[8]	eval-error:0.255208	train-error:0.131944
[9]	eval-error:0.270833	train-error:0.133681


## 预估器方式

In [14]:
import numpy as np
import pandas as pd
import pickle
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib

/Users/xiaonaozhong/anaconda3/envs/p3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [15]:
data=pd.read_csv('data/Pima-Indians-Diabetes.csv')

In [16]:
train,test=train_test_split(data)

feature_columns=['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction']
target_column='Outcome'


train_x=train[feature_columns].values
train_y=train[target_column].values
test_x=test[feature_columns].values
test_y=test[target_column].values


In [17]:
xgb_classifier=xgb.XGBClassifier(n_estimators=20,max_depth=4,learning_rate=0.1,subsample=0.7,colsample_bytree=0.7)

In [19]:
xgb_classifier.fit(train_x,train_y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=4,
              min_child_weight=1, missing=None, n_estimators=20, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.7, verbosity=1)

In [20]:
preds=xgb_classifier.predict(test_x)

In [21]:
((preds!=test_y).sum())/float(test_y.shape[0])

0.2708333333333333

## 内置的交叉验证与高级功能

In [22]:
xgb.cv(param,dtrain,num_round,nfold=5,metrics={'error'},seed=0)

,train-error-mean,train-error-std,test-error-mean,test-error-std
0,0.005144,0.000506,0.005373,0.002002
1,0.007715,0.003986,0.007830,0.004074
2,0.003071,0.001521,0.003377,0.001790
3,0.002380,0.000960,0.002150,0.001321
4,0.001382,0.000372,0.001535,0.000971
5,0.001497,0.000372,0.001535,0.000971
6,0.001420,0.000335,0.001382,0.000895
7,0.001459,0.000312,0.001382,0.001128
8,0.001459,0.000312,0.001535,0.000971
9,0.001459,0.000312,0.001382,0.001128


In [24]:
def fpreproc(dtrain,dtest,param):
    label=dtrain.get_label()
    ratio=float(np.sum(label==0))/np.sum(label==1)
    param['scale_pos_weight']=ratio
    return (dtrain,dtest,param)

xgb.cv(param,dtrain,num_round,nfold=5,metrics={'auc'},seed=0,fpreproc=fpreproc)

,train-auc-mean,train-auc-std,test-auc-mean,test-auc-std
0,0.995888,0.000420,0.995651,0.001985
1,0.998347,0.001219,0.997387,0.001510
2,0.999142,0.000618,0.998623,0.001357
3,0.999724,0.000298,0.999477,0.000924
4,0.999858,0.000238,0.999481,0.000926
5,0.999830,0.000294,0.999342,0.001202
6,0.999853,0.000238,0.999468,0.000922
7,0.999853,0.000238,0.999497,0.000932
8,0.999971,0.000045,0.999944,0.000069
9,0.999986,0.000021,0.999971,0.000036


## 自定义损失函数和评估准则

In [25]:
#自定义的时候，需要提供损失函数的一阶导和二阶导
def logregobj(preds,dtrain):
    labels=dtrain.get_label()
    preds=1.0/(1.0+np.exp(-preds))
    grad=preds-labels
    hess=preds*(1.0-preds)
    return grad,hess

In [26]:
def evalerror(preds,dtrain):
    labels=dtrain.get_label()
    return 'error',float(sum(labels!=(preds>0.0)))/len(labels)

In [28]:
watchlist=[(dtest,'eval'),(dtrain,'train')]
param={'max_depth':3,'eta':0.1,'silent':1}
num_round=5

In [29]:
bst=xgb.train(param,dtrain,num_round,watchlist,logregobj,evalerror)

[0]	eval-rmse:0.306902	train-rmse:0.306163	eval-error:0.518312	train-error:0.517887
[1]	eval-rmse:0.17919	train-rmse:0.177276	eval-error:0.518312	train-error:0.517887
[2]	eval-rmse:0.172566	train-rmse:0.171727	eval-error:0.016139	train-error:0.014433
[3]	eval-rmse:0.269611	train-rmse:0.271113	eval-error:0.016139	train-error:0.014433
[4]	eval-rmse:0.396904	train-rmse:0.398245	eval-error:0.016139	train-error:0.014433


In [30]:
xgb.cv(param,dtrain,num_round,nfold=5,seed=0,obj=logregobj,feval=evalerror)

,train-error-mean,train-error-std,train-rmse-mean,train-rmse-std,test-error-mean,test-error-std,test-rmse-mean,test-rmse-std
0,0.517887,0.001085,0.308880,0.005170,0.517886,0.004343,0.309038,0.005207
1,0.517887,0.001085,0.176504,0.002046,0.517886,0.004343,0.177802,0.003767
2,0.014433,0.000223,0.172680,0.003719,0.014433,0.000892,0.174890,0.009391
3,0.014433,0.000223,0.275761,0.001776,0.014433,0.000892,0.276689,0.005918
4,0.014433,0.000223,0.399889,0.003369,0.014433,0.000892,0.400118,0.006243


In [32]:
preds=bst.predict(dtest,ntree_limit=1)

In [33]:
preds

array([0.23779708, 0.65025556, 0.23779708, ..., 0.65025556, 0.23679176,
       0.65025556], dtype=float32)

In [34]:
preds=bst.predict(dtest,ntree_limit=9)
preds

array([-0.35670054,  0.6696481 , -0.35670054, ...,  1.2099813 ,
       -0.5371261 ,  1.2099813 ], dtype=float32)

## 预估器sklearn与xgboost配合

In [36]:
import pickle
import xgboost as xbg
import numpy as np
from sklearn.model_selection import KFold,train_test_split,GridSearchCV
from sklearn.metrics import confusion_matrix,mean_squared_error
from sklearn.datasets import load_iris,load_digits,load_boston

In [37]:
rng=np.random.RandomState(31337)

In [38]:
digits=load_digits(2)
y=digits['target']
x=digits['data']

kf=KFold(n_splits=2,shuffle=True,random_state=rng)

In [39]:
for train_index,test_index in kf.split(x):
    xgb_model=xgb.XGBClassifier().fit(x[train_index],y[train_index])
    predictions=xgb_model.predict(x[test_index])
    actuals=y[test_index]
    print(confusion_matrix(actuals,predictions))

[[87  0]
 [ 1 92]]
[[91  0]
 [ 3 86]]


In [40]:
digits=load_iris()
y=digits['target']
x=digits['data']

kf=KFold(n_splits=2,shuffle=True,random_state=rng)
for train_index,test_index in kf.split(x):
    xgb_model=xgb.XGBClassifier().fit(x[train_index],y[train_index])
    predictions=xgb_model.predict(x[test_index])
    actuals=y[test_index]
    print(confusion_matrix(actuals,predictions))

[[19  0  0]
 [ 0 31  3]
 [ 0  1 21]]
[[31  0  0]
 [ 0 16  0]
 [ 0  3 25]]


## 网格搜索交叉验证

In [41]:
boston=load_boston()
y=boston['target']
x=boston['data']

xgb_model=xgb.XGBRegressor()

clf=GridSearchCV(xgb_model,{'max_depth':[2,4,6],'n_estimators':[50,100,200]},verbose=1)

In [42]:
clf.fit(x,y)

/Users/xiaonaozhong/anaconda3/envs/p3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 9 candidates, totalling 27 fits
[10:09:48] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:09:48] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:09:48] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:09:48] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:09:48] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:09:48] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:09:48] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:09:48] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:09:48] W

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:    1.3s finished
/Users/xiaonaozhong/anaconda3/envs/p3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bynode=1,
                                    colsample_bytree=1, gamma=0,
                                    importance_type='gain', learning_rate=0.1,
                                    max_delta_step=0, max_depth=3,
                                    min_child_weight=1, missing=None,
                                    n_estimators=100, n_jobs=1, nthread=None,
                                    objective='reg:linear', random_state=0,
                                    reg_alpha=0, reg_lambda=1,
                                    scale_pos_weight=1, seed=None, silent=None,
                                    subsample=1, verbosity=1),
             iid='warn', n_jobs=None,
             param_grid={'max_depth': [2, 4, 6],
                         'n_estimators': [50, 100, 200]},
          

In [43]:
clf.best_score_

0.6001029721598573

In [44]:
clf.best_params_

{'max_depth': 4, 'n_estimators': 100}

## early-stopping

In [49]:
digits=load_digits(2)
x=digits['data']
y=digits['target']

x_train,x_val,y_train,y_val=train_test_split(x,y,random_state=0)
clf=xgb.XGBClassifier()
clf.fit(x_train,y_train,early_stopping_rounds=10,eval_metric='auc',eval_set=[(x_val,y_val)])

[0]	validation_0-auc:0.999497
Will train until validation_0-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.999497
[2]	validation_0-auc:0.999497
[3]	validation_0-auc:0.999749
[4]	validation_0-auc:0.999749
[5]	validation_0-auc:0.999749
[6]	validation_0-auc:0.999749
[7]	validation_0-auc:0.999749
[8]	validation_0-auc:0.999749
[9]	validation_0-auc:0.999749
[10]	validation_0-auc:1
[11]	validation_0-auc:1
[12]	validation_0-auc:1
[13]	validation_0-auc:1
[14]	validation_0-auc:1
[15]	validation_0-auc:1
[16]	validation_0-auc:1
[17]	validation_0-auc:1
[18]	validation_0-auc:1
[19]	validation_0-auc:1
[20]	validation_0-auc:1
Stopping. Best iteration:
[10]	validation_0-auc:1



XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

## 特征重要度

In [50]:
iris=load_iris()
y=iris['target']
x=iris['data']

xgb_model=xgb.XGBClassifier().fit(x,y)

In [51]:
feature_names=['sepal_length','sepal_width','petal_length','petal_width']
feature_importances=xgb_model.feature_importances_

In [54]:
feature_importances

array([0.01251974, 0.03348068, 0.59583396, 0.35816565], dtype=float32)

In [52]:
indices=np.argsort(feature_importances)[::-1]

In [53]:
indices

array([2, 3, 1, 0])

In [56]:
for index in indices :
    print((feature_names[index],feature_importances[index]))

('petal_length', 0.59583396)
('petal_width', 0.35816565)
('sepal_width', 0.033480678)
('sepal_length', 0.012519742)
